## Unpackage Json to COCO Formatt

In [1]:
import json

# input and output paths
# (change as needed based on your project folder structure)
input_path = 'data/train_annotations.json'
output_path = 'data/train_annotations_coco.json'

# read train annotations data
with open(input_path) as f:
    train_annotations = json.load(f)

# initialize COCO data structure
coco_data = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "individual_tree", "supercategory": "tree"},
        {"id": 2, "name": "group_of_trees", "supercategory": "tree"},
    ]
}

# category mapping
category_map = {
    "individual_tree": 1,
    "group_of_trees": 2
}

# initialize annotation and image ID counters
annotation_id = 1
image_id = 1

# for each image...
for image in train_annotations["images"]:
    
    # add image metadata
    coco_data["images"].append(
        {
            "id": image_id,
            "file_name": image["file_name"],
            "width": image["width"],
            "height": image["height"]
        }
    )

    # for each annotation in this image
    for ann in image.get("annotations", []):
        # extract segmentation polygon
        segmentation = ann["segmentation"]

        # skip if fewer than 3 points (expected to cause errors later)
        if len(segmentation) < 6:
            continue

        # append annotation
        coco_data["annotations"].append(
            {
                "id": annotation_id,                                    # annotation ID
                "image_id": image_id,                                   # image ID
                "category_id": category_map[ann["class"]],              # category ID
                "segmentation": [segmentation],                         # segmentation polygon
                "area": 0,                                              # area (not used but setting anyway)
                "bbox": [                                               # bounding box
                    min(segmentation[::2]),                             # ... x
                    min(segmentation[1::2]),                            # ... y
                    max(segmentation[::2]) - min(segmentation[::2]),    # ... w
                    max(segmentation[1::2]) - min(segmentation[1::2])   # ... h
                ],
                "iscrowd": 0,                                           # is-crowded (not used but setting anyway)
                "score": ann.get("confidence_score", 1.0)               # confidence score (nonsense for ground-truth but setting anyway)
            }
        )

        # increment annotation ID counter
        annotation_id += 1

    # increment image ID counter
    image_id += 1

# save output
with open(output_path, "w") as f:
    json.dump(coco_data, f, indent=2)

## Unpackage COCO Json to labels

In [6]:
#!/ucd
#sr/bin/env python3
import os, json, sys
from collections import defaultdict

# ---- CONFIG (edit paths if different) ----
input_json = os.path.join("data", "train_annotations_coco.json")
ROOT = os.path.join("data", "model-data")  # contains train/ valid/ test/
SPLITS = ["train", "valid", "test"]        # change if you only use some

In [7]:
with open(input_json) as f:
    COCO_JSON = json.load(f)

COCO_JSON

{'images': [{'id': 1,
   'file_name': '10cm_train_1.tif',
   'width': 1024,
   'height': 1024},
  {'id': 2, 'file_name': '10cm_train_10.tif', 'width': 1024, 'height': 1024},
  {'id': 3, 'file_name': '10cm_train_11.tif', 'width': 1024, 'height': 1024},
  {'id': 4, 'file_name': '10cm_train_12.tif', 'width': 1024, 'height': 1024},
  {'id': 5, 'file_name': '10cm_train_13.tif', 'width': 1024, 'height': 1024},
  {'id': 6, 'file_name': '10cm_train_14.tif', 'width': 1024, 'height': 1024},
  {'id': 7, 'file_name': '10cm_train_15.tif', 'width': 1024, 'height': 1024},
  {'id': 8, 'file_name': '10cm_train_16.tif', 'width': 1024, 'height': 1024},
  {'id': 9, 'file_name': '10cm_train_17.tif', 'width': 1024, 'height': 1024},
  {'id': 10, 'file_name': '10cm_train_18.tif', 'width': 1024, 'height': 1024},
  {'id': 11, 'file_name': '10cm_train_19.tif', 'width': 1024, 'height': 1024},
  {'id': 12, 'file_name': '10cm_train_2.tif', 'width': 1024, 'height': 1024},
  {'id': 13, 'file_name': '10cm_train_20.tif

### Helper: normalize polygon pairs
- COCO polygon segmentations are flat lists of pixel coordinates: [x1, y1, x2, y2, …].

- YOLO wants normalized coordinates (divide by image width/height).

- This generator yields normalized (x, y) pairs ready to be written.

#### Process

- Takes a flat list of coordinates: [x1, y1, x2, y2, ...]

- Groups them into (x, y) pairs using zip(it, it)

- Divides each coordinate by the image width (w) and height (h) to normalize them between 0 and 1

- This is standard in YOLO and most deep learning frameworks — so coordinates are relative, not pixel-based.

In [8]:
def norm_pair_iter(coords, w, h):
    """Yield (x,y) normalized to [0,1] from a flat list of pixel coords."""
    it = iter(coords)            # coords like [x1,y1,x2,y2,...]
    for x, y in zip(it, it):     # step through pairs
        yield (float(x)/w, float(y)/h)


### Helper: decide which split an image belongs to

- We look for the image file under each split’s images/.

- Whatever split contains the image determines where we write the label file.

- This lets you reuse a single COCO file for all splits.

In [ ]:
def find_split_for_file(filename):
    """Return 'train'/'valid'/'test' if the image exists in that split, else None."""
    for s in SPLITS:
        p = os.path.join(ROOT, s, "images", filename)
        if os.path.exists(p):
            return s
    return None




### main() – the pipeline

- Safety check + load the COCO dict. The standard COCO structure is:

    - images: list of image dicts (id, file_name, width, height, …)

    - annotations: list of annotation dicts (image_id, category_id, bbox, segmentation, …)

    - categories: list of category dicts (id, name, …)

In [10]:
def main():
    if not os.path.exists(COCO_JSON):
        sys.exit(f"COCO file not found: {COCO_JSON}")

    with open(COCO_JSON, "r") as f:
        coco = json.load(f)


### Build fast lookups & category mapping

- Why mapping? COCO category_id numbers are often not 0..N-1. YOLO expects classes to be 0..nc-1.

- We sort the COCO category ids, then assign 0,1,2,… accordingly.

- names keeps the readable class names in that order.

(You’ll often also build anns_by_img = defaultdict(list) and fill it by iterating over coco["annotations"] so you can fetch annotations per image fast.)

In [11]:
# id -> image dict (quick access)
images_by_id = {img["id"]: img for img in coco.get("images", [])}

# map arbitrary COCO category ids to contiguous [0..nc-1]
cat_ids_sorted = sorted(c["id"] for c in coco.get("categories", []))
cat_to_yolo = {cid: i for i, cid in enumerate(cat_ids_sorted)}

# list of class names, ordered by the mapping above (for data.yaml later)
names = [next(c["name"] for c in coco["categories"] if c["id"] == cid)
         for cid in cat_ids_sorted]


NameError: name 'coco' is not defined

### Make sure labels/ folders exist

YOLO expects one .txt file per image in a sibling labels/ folder.

This creates:


data/model-data/
  ├── train/
  │   ├── images/
  │   └── labels/
  ├── valid/
  │   ├── images/
  │   └── labels/
  └── test/
      ├── images/
      └── labels/


In [12]:
for s in SPLITS:
    os.makedirs(os.path.join(ROOT, s, "labels"), exist_ok=True)


### Iterate images → write labels

(You’ll see this in the lower part of the script)

- COCO bbox = [x_min, y_min, width, height] (pixels)

- YOLO bbox = class cx cy w h normalized to 0..1 (center-based)

- YOLOv8 segmentation = class x1 y1 x2 y2 … (normalized, one polygon per line)

Typical loop structure:

In [ ]:
counts = defaultdict(int)  # stats

# optional: group annotations by image id for speed
annotations_by_img = defaultdict(list)
for annotation in coco.get("annotations", []):
    annotations_by_img[annotation["image_id"]].append(annotation)

for img in coco.get("images", []):
    filename = img["file_name"]
    w, h     = img["width"], img["height"]

    split = find_split_for_file(filename)
    if split is None:
        counts["skipped_no_split"] += 1
        continue

    label_path = os.path.join(ROOT, split, "labels",
                              os.path.splitext(filename)[0] + ".txt")

    lines = []  # lines to write for this image

    for annotation in annotations_by_img.get(img["id"], []):
        # skip crowd/RLE etc if you only want polygons or bboxes
        if annotation.get("iscrowd", 0) == 1:
            counts["skipped_crowd"] += 1
            continue

        categoryid = annotation["category_id"]
        cls = cat_to_yolo[categoryid]  # contiguous class id

        # # ---- Option A: bbox → YOLO box ----
        # if "bbox" in annotation and annotation["bbox"]:
        #     x, y, bw, bh = annotation["bbox"]         # COCO = top-left + width/height
        #     xc = (x + bw / 2.0) / w            # YOLO needs center-x, center-y, w, h
        #     yc = (y + bh / 2.0) / h
        #     ww = bw / w
        #     hh = bh / h

        #     # clip to [0,1] just in case
        #     xc = min(max(xc, 0), 1); yc = min(max(yc, 0), 1)
        #     ww = min(max(ww, 0), 1); hh = min(max(hh, 0), 1)

        #     lines.append(f"{cls} {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}")



        # ---- Option B: polygon segmentations → YOLOv8 segmentation ----
        # YOLOv8 expects: cls x1 y1 x2 y2 ... (normalized), one polygon per line
        segmentations = annotation.get("segmentations", [])
        if segmentations and isinstance(segmentations, list):   # skip RLE (dict) here
            for segmentation in segmentations:
                if len(segmentation) >= 6 and len(segmentation) % 2 == 0:
                    pts = [f"{x:.6f} {y:.6f}" for x, y in norm_pair_iter(segmentation, w, h)]
                    lines.append(f"{cls} " + " ".join(pts))
                else:
                    counts["skipped_bad_seg"] += 1

    # write all objects for this image
    with open(label_path, "w") as f:
        f.write("\n".join(lines))

    counts["images_processed"] += 1

# (Optional) Print summary, write data.yaml using `names` etc.


NameError: name 'coco' is not defined

### Why this structure works well

- Split detection via find_split_for_file lets you use one COCO file for all splits.

- Contiguous class ids guarantee YOLO training scripts don’t choke on sparse ids.

- One label file per image matches Ultralytics’ expected structure.

- Normalization is done once and always with actual image width/height.

- Polygon → YOLOv8 handled by norm_pair_iter.

{'path': '/Users/mitchellpalmer/Projects/solafune-canopy-capstone-clean/data/model-data', 'train': 'data/model-data/train/images', 'val': 'data/model-data/valid/images', 'test': 'data/model-data/test/images', 'names': ['individual_tree', 'group_of_trees']}
